In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV,StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import tensorflow as tf

## **Reading the data**

In [ ]:
Train = pd.read_csv("../input/dont-overfit-ii/train.csv")
Test = pd.read_csv("../input/dont-overfit-ii/test.csv")

## **Exploring the data**

In [ ]:
Train.head()

In [ ]:
Test.head()

In [ ]:
# Column names of the training data
Train.columns

In [ ]:
# Column names of the test data
Test.columns

#### Discovering the shape and the data types of the data

In [ ]:
# Print the information of the train data 
print("Train data information:")
print(Train.info(), end = '\n\n')
print("Shape:", Train.shape, end = '\n\n\n', sep = '\t')


# Print the information of the test data 
print("Test data information:")
print(Test.info(), end = '\n\n')
print("Shape:", Test.shape, end = '\n', sep = '\t')


#### Summary statistics

In [ ]:
Train.describe()

In [ ]:
Test.describe()

#### Check the missing values

In [ ]:
Train_Nulls = []
for col in Train.columns:
    Train_Nulls.append(Train[col].isnull().sum())

print(Train_Nulls)

In [ ]:
Test_Nulls = []
for col in Test.columns:
    Test_Nulls.append(Test[col].isnull().sum())

print(Test_Nulls)

#### **No null values** in both the train and the test data

#### Check the duplicates

In [ ]:
Train.duplicated().sum()

In [ ]:
Test.duplicated().sum()

#### **No duplicates** in both the train and the test data

## **EDA**

In [ ]:
# Specify the predictors
predictors = list(Train.columns) 
predictors.remove('target')
sns.pairplot(data = Train,x_vars=predictors[1:10], y_vars=predictors[1:10], hue = 'target', diag_kind = 'hist')

In [ ]:
# Regression plot
sns.regplot(x='0',y='target',data = Train)

## Feature selection

In [ ]:
def getCorr(x,y):
    corr, _ = scipy.stats.pearsonr(x, y) 
    return corr

In [ ]:
detailed_features=[]
for feature in range(300):
    sns.regplot(x=str(feature),y='target',data = Train)
    detailed_features.append({'feature':feature,'slope':getCorr(Train['target'],Train[str(feature)])})

In [ ]:
def getSlope(df):
    return abs(df['slope'])

In [ ]:
detailed_features.sort(key=getSlope,reverse=True)

In [ ]:
features_to_save=[]
for iteration in range(20):
    features_to_save.append(detailed_features[iteration]['feature'])

In [ ]:
def getDfProcessed(df_toProcess,features_to_save):
    df_processed = pd.DataFrame() 
    for iteration in features_to_save:
        feature = df_toProcess[str(iteration)].values
        df_processed[str(iteration)] = feature 
    return df_processed

In [ ]:
dftrain_processed = getDfProcessed(Train,features_to_save)

In [ ]:
dftest_processed = getDfProcessed(Test,features_to_save)

### Data preparation and splitting

In [ ]:
# Isolate the taeget column
Y = Train['target']

# Extract the train features
X = dftrain_processed.copy()
# X.drop(['id', 'target'], axis = 1, inplace = True)

In [ ]:
# Test.drop(['id'], axis = 1, inplace = True)

In [ ]:
# Features scaling
Scaler = MinMaxScaler()
X = pd.DataFrame(Scaler.fit_transform(X))
Test_1 = dftest_processed.copy()
Test_1 = pd.DataFrame(Scaler.fit_transform(Test_1))

In [ ]:
# [Train validation] splitting
X_Train, X_Valid, Y_Train, Y_Valid = train_test_split(X,Y, train_size=0.8, test_size=0.2,random_state=0)

## Grid search

In [ ]:
# from sklearn.metrics import roc_auc_score
# from sklearn.linear_model import SGDClassifier
# from sklearn import tree
# from sklearn.svm import SVC
# from sklearn.neighbors import NearestCentroid
# from sklearn.naive_bayes import GaussianNB
# # # Intiate the model
# # naivebayes = GaussianNB()
# # # Fit the model
# # naivebayes.fit(X_train, y_train)

In [ ]:
# Models = [GaussianNB(), RandomForestClassifier(),LogisticRegression(),SGDClassifier(),tree.DecisionTreeClassifier(),NearestCentroid(),SVC()]

# ModelNames = ['GaussianNB', 'RandomForestclassifier','LogisticRegression','SGDClassifier','DecisionTreeClassifier','NearestCentroid','SVC']
# AUC = []
# Model_AUC = {}

# for model in Models:
#     # Intialize the model
#     Classifier = model
#     # Fitting
#     Classifier.fit(X_Train,Y_Train)
# #     print("model_name : ",model, end = '\n')
# #     print(Classifier.get_params(), end = '\n\n\n')
    
#     # Prediction
#     Valid_Prediction = Classifier.predict(X_Valid)
    
#     # ROC
#     auc = roc_auc_score(Y_Valid, Valid_Prediction)
#     AUC.append(auc)


# # Result    
# Model_AUC = {'Classifier' : ModelNames, 'AUC': AUC}
# Model_AUC_df = pd.DataFrame(Model_AUC)  
# Model_AUC_df

## **Models**

#### **1.** LogisticRegression

In [ ]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# fit the model with data
logreg.fit(X_Train, Y_Train)

# Predict the test values
y_pred = logreg.predict(X_Valid)

In [ ]:
logreg.classes_

In [ ]:
# logreg.coef_

In [ ]:
#### Model Evaluation
# import the metrics class
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(Y_Valid, y_pred))
print("Precision:",metrics.precision_score(Y_Valid, y_pred))
print("Recall:",metrics.recall_score(Y_Valid, y_pred))

In [ ]:
# Let's call the confusion matrix method
cnf_matrix = metrics.confusion_matrix(Y_Valid, y_pred)
cnf_matrix

In [ ]:
# Now, we will try to display it in a nicer way.
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
y_pred_proba = logreg.predict_proba(X_Valid)[::,1]
fpr, tpr, _ = metrics.roc_curve(Y_Valid,  y_pred_proba)
auc = metrics.roc_auc_score(Y_Valid,  y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(logreg, X, Y, cv=4, return_train_score = True, scoring=('accuracy', 'precision', 'recall', 'f1'))
# cv_results

In [ ]:
cv_df = pd.DataFrame(cv_results)
cv_df

In [ ]:
# Let's create our function that can change the threshold
def predict_by_threshold(model, test_set , threshold):
    import numpy as np
    preds = np.where(model.predict_proba(test_set)[:,1] > threshold, 1, 0)
    return preds 

In [ ]:
# Let's see the output if the threshold was 0.7
y_pred_th = predict_by_threshold(logreg, X_Valid, 0.8) 

In [ ]:
print("Accuracy:",metrics.accuracy_score(Y_Valid, y_pred_th))
print("Precision:",metrics.precision_score(Y_Valid, y_pred_th))
print("Recall:",metrics.recall_score(Y_Valid, y_pred_th))

In [ ]:
logreg_predictions = logreg.predict(Test_1)

In [ ]:
dfsubmission = pd.DataFrame() 
dfsubmission['id'] = Test['id'] 
dfsubmission['target'] = logreg_predictions

In [ ]:
dfsubmission

In [ ]:
dfsubmission.to_csv('submission.csv', index=False) 

#### 2. Neural Network

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(7, activation=tf.nn.relu),  
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X, Y, epochs=10)

In [ ]:
predictions_list = list(model.predict(Test_1))

In [ ]:
def getPredictions(prediction_list):
    prediction_list_processced = []
    for iteration in prediction_list :
        prediction_list_processced.append(int(round(iteration[1])))
    return prediction_list_processced

In [ ]:
dfsubmission_2 = pd.DataFrame() 
dfsubmission_2['id'] = Test['id'] 
dfsubmission_2['target'] = getPredictions(predictions_list)

In [ ]:
dfsubmission_2

In [ ]:
dfsubmission_2.to_csv('submission_2.csv', index=False) 

#### 3. GaussianNB

In [ ]:
# Import Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
# Intiate the model
naivebayes = GaussianNB()
# Fit the model
naivebayes.fit(X_Train, Y_Train)


In [ ]:
cv_results = pd.DataFrame(cross_validate(naivebayes, X, Y, cv=4, return_train_score = True, scoring=('accuracy', 'precision', 'recall', 'f1')))
cv_results

In [ ]:
# Adjust display presion
pd.set_option("display.precision", 8)

# Using of predict 
naivebayes_predict = naivebayes.predict(X_Valid)
# Using of predict_proba
naivebayes_predict_proba = naivebayes.predict_proba(X_Valid)
# Generate a Dataframe
results = pd.DataFrame({'Actual': Y_Valid.tolist(), 
                      'Predicted Class': naivebayes_predict.tolist(), 
                      'Predicted Class Probapitiy': naivebayes_predict_proba[:, 1].tolist()})

In [ ]:
# Print the head of the Actual vs. Predicted DataFrame
results.head(10)

In [ ]:
naivebayes_predictions = naivebayes.predict(Test_1)

In [ ]:
dfsubmission_3 = pd.DataFrame() 
dfsubmission_3['id'] = Test['id'] 
dfsubmission_3['target'] = naivebayes_predictions

In [ ]:
dfsubmission_3

In [ ]:
dfsubmission_3.to_csv('submission_3_.csv', index=False) 

#### 4. DecisionTreeClassifier

In [ ]:
# Import the Model 
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Intiate the model
dtree = DecisionTreeClassifier()

# Fitting the Model 
dtree.fit(X_Train, Y_Train)

# Predict for testing
dtree_predict = dtree.predict(X_Valid)

# Print the accuracy
print('Decision Tree accuracy: ', metrics.accuracy_score(Y_Valid, dtree_predict))


In [ ]:
# Let's call the confusion matrix method
cnf_matrix = metrics.confusion_matrix(Y_Valid, dtree_predict)


class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
cv_results = pd.DataFrame(cross_validate(dtree, X, Y, cv=4, return_train_score = True, scoring=('accuracy', 'precision', 'recall', 'f1')))
cv_results

In [ ]:
dtree_predictions = dtree.predict(Test_1)

In [ ]:
dfsubmission_4 = pd.DataFrame() 
dfsubmission_4['id'] = Test['id'] 
dfsubmission_4['target'] = dtree_predictions

In [ ]:
dfsubmission_4.to_csv('submission_4.csv', index=False) 

#### 5. RandomForestClassifier

In [ ]:
# Import Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
# Intiate the model
random_forset = RandomForestClassifier(n_estimators=100)
# Fit the Model
random_forset.fit(X_Train, Y_Train) 
# Predict for testing
random_forset_predict = random_forset.predict(X_Valid)
# Let's see the perforamce of our Model using the Cross Validation Method
cv_results_random_forest = pd.DataFrame(cross_validate(random_forset, X, Y, cv=4, return_train_score = True, scoring=('accuracy', 'precision', 'recall', 'f1')))
# View the results
cv_results_random_forest.head()

#### 6. ExtraTreesClassifier

In [ ]:
# Import ExtraTrees Classifier
from sklearn.ensemble import ExtraTreesClassifier
# Intiate the model
extra_tree = ExtraTreesClassifier(n_estimators=100)
# Fit the Model
extra_tree.fit(X_Train, Y_Train) 
# Predict for testing
extra_tree_predict = extra_tree.predict(X_Valid)
# Let's see the perforamce of our Model using the Cross Validation Method
cv_results_extra_tree = pd.DataFrame(cross_validate(extra_tree, X, Y, cv=4, return_train_score = True, scoring=('accuracy', 'precision', 'recall', 'f1')))
# View the results
cv_results_extra_tree.head()